In [382]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

In [383]:
data = load_diabetes()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y[0]

151.0

In [384]:
X_train_num = np.dot(X_train,100).astype(int)
X_test_num = np.dot(X_test,100).astype(int)
y_train_num = y_train
y_test_num = y_test

In [385]:
X_train_den = np.full(X_train.shape, 100) 
X_test_den = np.full(X_test.shape, 100) 
y_train_den = np.ones(len(y_train))
y_test_den = np.ones(len(y_test))

In [430]:
# Python Program to find the L.C.M. of two input number
def compute_lcm(x, y):
   # choose the greater number 
    if (x > y):
        greater = x
    else:
        greater = y
    while(True):
        if(x==0):
            lcm = y
            break
        elif(y==0):
            lcm = x
            break
        elif((greater % x == 0) and (greater % y == 0)):
            lcm = greater
            break
        greater += 1
    return lcm
# multiplying a/b and c/d
def multiply(a,b,c,d): # of the form a/b * c/d
    num = a*c
    den = b*d
    return num,den
#adding num and den version of the form a/b and c/d
def addition(a,b,c,d):
    den = compute_lcm(b,d)
#     den = b*d
    if b ==0:
        num = c ; den = d
    elif d==0:
        num= a ; den = b
    else:    
        num = ((den/b)*a) + ((den/d)*c)
#     print(num,den)
    return float(num),float(den)

#subtraction between a/b and c/d
def subtraction(a,b,c,d):
    den = compute_lcm(b,d)
    if b ==0:
        num = c ; den = d
    elif d==0:
        num= a ; den = b
    else:    
        num = ((den/b)*a) - ((den/d)*c)
    return num,den
    

#claculate y_hat =wx+b | x is in the form a single array and w is in the form of array too
def y_hat(w,x_num,x_den,b):
    num=0; den =0
#     print(len(w))
    #multiplying w and x
    for i in range (len(w)):
#         print(w[i][0],x_num[i])
        mul_num = w[i][0] * x_num[i]
#         print(w[i][0],x_den[i])
        mul_den = w[i][1] * x_den[i]
#         print(mul_num,mul_den)
        num,den = addition(num,den,mul_num,mul_den)
#         print(num,den,mul_num,mul_den)
    #adding with bias
    num,den = addition(num,den,b[0],b[1]) 
#     print(num,den)
    return num,den    
#calculating (y-y_hat)**2
def mse(y_hat_num,y_hat_den,y_num,y_den,n):
    num=0;den =0
    for i in range(len(y_hat_num)):
        num1,den1=subtraction(y_num[i],y_den[i],y_hat_num[i],y_hat_den[i])
        num1 = num1*num1
        den1 = den1*den1
        num,den = addition(num,den,num1,den1)
    den = den * n
    print(num,den)
    return num,den

#calculating subtraction of Y_hat Y
def y_hat_substraction(y_hat_num,y_hat_den,y_num,y_den):
    y_diff =np.zeros((y_hat_num.shape[0],2))
    for i in range(len(y_hat_num)):
        y_diff[i][0],y_diff[i][1] = subtraction(y_hat_num[i],y_hat_den[i], y_num[i], y_den[i])
    return y_diff

#calculating multiplication of subtraction and X
def delta_w(y_diff,x_num,x_den,n):
    sum_num=0;sum_den=0;wnum=np.zeros(x_num.shape[1]);wden=np.zeros(x_num.shape[1])
    x_num = x_num.T
    x_den = x_den.T
    for i in range(x_num.shape[0]):
        for j in range (y_diff.shape[1]):
            z_num = x_num[j]
            z_den = x_den[j] 
            num,den = multiply(z_num[j],z_den[j],y_diff[j][0], y_diff[j][1])    
            sum_num,sum_den = addition(num,den,sum_num,sum_den)
        delta_w_num = sum_num*2
        delta_w_den = sum_den*n
        wnum[i]=sum_num
        wden[i]=sum_den
    print(wnum,wden)
    return wnum,wden
def delta_b(y):
    num=0;den=0
    for i in range (y.shape[0]):
        num,den = addition(num,den,y[i][0], y[i][1])
    num = num *2
    den = den*y.shape[0]
    return num,den

def get_new_deltas(lr, dw_num,dw_den,db_num,db_den,w_num,w_den,b_num,b_den):
#     wtemp_num = lr[0] * dw_num
#     wtemp_den = lr[1] * dw_den
#     btemp_num = lr[0] * db_num
#     btemp_den = lr[1] * db_den 
#     dw_num,dw_den = subtraction(w_num,w_den,wtemp_num,wtemp_den)
#     db_num,db_den = subtraction(b_num,b_den,btemp_num,btemp_den)
    
    for i in range(len(dw_num)):
        dw_num[i]= dw_num[i]*lr[0]
        dw_den[i]= dw_den[i]*lr[1]
    btemp_num = lr[0] * db_num
    btemp_den = lr[1] * db_den 
    
    for i in range(len(dw_num)):
        dw_num[i],dw_den[i] = subtraction(w_num,w_den,dw_num[i],dw_den[i]) 
    db_num,db_den = subtraction(b_num,b_den,btemp_num,btemp_den)
    return  dw_num,dw_den,db_num,db_den

In [431]:
w = np.zeros((X_train.shape[1],2))
b = np.zeros(2)


In [432]:
# finding y_hat 
y_hat_num = np.zeros(len(y_train_num))
y_hat_den = np.zeros(len(y_train_num))
for i in range(y_train_num.shape[0]):
    y_hat_num[i], y_hat_den[i] = y_hat(w,X_train_num[i],X_train_den[i],b)

In [433]:
#get the MSE
n = len(y_train_num)
mse_num,mse_den = mse(y_hat_num,y_hat_den,y_train_num,y_train_den,n)
print(mse_num,mse_den)

10488097.0 353.0
10488097.0 353.0


In [434]:
#get y_hat subtraction
y_hat_diff = np.zeros((len(y_train_num),2))
y_hat_diff =  y_hat_substraction(y_hat_num,y_hat_den,y_train_num,y_train_den)


In [435]:
y_hat_diff.shape[0]

353

In [436]:
print(X_train_num[0])

[ 7  5  1  5  3  4 -3  3  2  0]


In [437]:
#get delta_w 
delta_w_num,delta_w_den = delta_w(y_hat_diff,X_train_num,X_train_den,y_hat_diff.shape[0])
# print(delta_w_num,delta_w_den)

[ 1758.  3516.  5274.  7032.  8790. 10548. 12306. 14064. 15822. 17580.] [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]


In [438]:
#get delta_b
delta_b_num,delta_b_den = delta_b(y_hat_diff)
print(delta_b_num,delta_b_den)

108538.0 353.0


In [439]:
lr = np.array([1,100])

In [440]:
#get new_delta
nw_num=nw_den=nb_num=nb_den =0
nw_num, nw_den,nb_num,nb_den = get_new_deltas(lr,delta_w_num,delta_w_den,delta_b_num,delta_b_den,nw_num,nw_den,nb_num,nb_den)
print(nw_num, nw_den,nb_num,nb_den)

[ 1758.  3516.  5274.  7032.  8790. 10548. 12306. 14064. 15822. 17580.] [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.] 108538.0 35300.0
